<a href="https://colab.research.google.com/github/mthavril/EDA_Penjualan_Kopi/blob/main/EDA_Penjualan_Kopi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**EDA Penjualan Kopi 2024–2025: Mengungkap Tren dan Perilaku Pelanggan**

## 1. Pemanggilan Dataset

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import os

os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/kaggle.json

In [ ]:
!kaggle datasets  download -d ihelon/coffee-sales

In [ ]:
import zipfile

with zipfile.ZipFile("coffee-sales.zip", "r") as zip_ref:
  zip_ref.extractall("coffee_sales_dataset")

In [ ]:
os.listdir("coffee_sales_dataset")

## 2. Menggabungkan kedua file csv menjadi 1 file

In [ ]:
import pandas as pd

#Baca dua file CSV
df1 = pd.read_csv("coffee_sales_dataset/index_1.csv")
df2 = pd.read_csv("coffee_sales_dataset/index_2.csv")

#Gabungkan keduanya
df = pd.concat([df1, df2], ignore_index=True)
print("Jumlah total data:", len(df))
df.head()


## 3. Membuat kolom baru (tahun, bulan, hari dan payment_method) &&& mengganti kolom (money) menjadi (total_price)

In [ ]:
print(df.columns)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df['date'] = pd.to_datetime(df['date'])

df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month_name()
df['day_name'] = df['date'].dt.day_name()


In [ ]:
df = df.rename(columns={'money': 'total_price'})

In [ ]:
import numpy as np

df['payment_method'] = np.where(
    df['card'].notnull(), 'Card',
    np.where(
        df['cash_type'].str.lower() == 'cash', 'Cash', 'Unknown'
    )
)


In [ ]:
df[['date', 'coffee_name', 'payment_method', 'total_price']].head()


In [ ]:
df.head()

## 4. Langkah 1: Menganalisis tren penjualan berdasarkan waktu (hari, bulan, tahun).

**1A. Apakah penjualan meningkat di akhir tahun?**

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Hitung total penjualan per bulan per tahun
monthly_sales = (
    df.groupby(['year', 'month'])['total_price']
    .sum()
    .reset_index()
)

# Mengurutkan bulan
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']
monthly_sales['month'] = pd.Categorical(monthly_sales['month'], categories=month_order, ordered=True)
monthly_sales = monthly_sales.sort_values(['year', 'month'])

# Visualisasi
plt.figure(figsize=(10,5))
for year in df['year'].unique():
  data = monthly_sales[monthly_sales['year'] == year]
  plt.plot(data['month'], data['total_price'], marker='o', label=year)

plt.title("Tren Penjualan per Bulan (2024 vs 2025)")
plt.xlabel("Bulan")
plt.ylabel("TotalPenjualan")
plt.legend(title="Tahun")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


**1B. hari apa penjualan paling tinggi (weekday vs weekend)?**

In [ ]:
# Menambahkan nama hari
df['day_name'] = df['date'].dt.day_name()

# Total penjualan per hari
weekday_sales = (
    df.groupby('day_name')['total_price']
    .sum()
    .reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
    .reset_index()
)

# Visualisasi
plt.figure(figsize=(8,4))
plt.bar(weekday_sales['day_name'], weekday_sales['total_price'])
plt.title("Total Penjualan per Hari")
plt.xlabel("Hari")
plt.ylabel("Total Penjualan")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

weekday_sales


In [ ]:
# Menambahkan kolom weekend untuk analisis pola hari
df['is_weekend'] = df['day_name'].isin(['Saturday', 'Sunday'])
weekend_sales = df.groupby('is_weekend')['total_price'].sum().reset_index()
weekend_sales['Kategori'] = weekend_sales['is_weekend'].map({True: 'Weekend', False: 'Weekday'})

plt.figure(figsize=(6,4))
plt.bar(weekend_sales['Kategori'], weekend_sales['total_price'], color=['#c68e17', '#8b4513'])
plt.title("Perbandingan Penjualan Weekday vs Weekend")
plt.xlabel("Kategori Hari")
plt.ylabel("Total Penjualan")
plt.tight_layout()
plt.show()


## 5. Langkah2:	Menganalisis produk terlaris tiap tahun.

**Jenis kopi apa yang paling sering dibeli?**

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Menghitung total penjualan tiap kopi per tahun
top_products = (
    df.groupby(['year', 'coffee_name'])['total_price']
    .sum()
    .reset_index()
    .sort_values(['year', 'total_price'], ascending=[True, False])
)

# Menampilkan Top 5 produk untuk setiap tahun
for year in df['year'].unique():
  print(f"\n Top 5 Produk Kopi Terlaris Tahun {year}:")
  top5 = top_products[top_products['year'] == year].head().reset_index(drop=True)
  print(top5[['coffee_name', 'total_price']])

  # Visualisasikan grafik batang untuk Top 5
  plt.figure(figsize=(7,4))
  plt.bar(top5['coffee_name'], top5['total_price'], color='tan')
  plt.title(f" Top 5 Kopi Terlaris Tahun {year}")
  plt.xlabel("Jenis Kopi")
  plt.ylabel("Total Penjualan")
  plt.xticks(rotation=45)
  plt.tight_layout()
  plt.show()




## 6. Langkah3: Melihat metode pembayaran paling sering digunakan. (dari semua tahun) kalo yang di langkah 4 perbandingan / masing-maisng tahun

**Apakah pelanggan lebih sering pakai tunai, kartu, atau e-wallet?**

In [ ]:
df['payment_method'] = df['payment_method'].str.title()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Jumlah transaksi per metode pembayaran (semua tahun)
payment_counts = (
    df.groupby('payment_method')['total_price']
    .agg(['count', 'sum'])
    .reset_index()
    .sort_values('count', ascending=False)
)


payment_counts.rename(columns={'count': 'jumlah_transaksi', 'sum': 'total_penjualan'}, inplace=True)

print("Ringkasan Metode Pembayaran (Semua Tahun):")
print(payment_counts)


# Visualisasi: Bar Chart (frekuensi transasksi)
plt.figure(figsize=(7,4))
plt.bar(payment_counts['payment_method'], payment_counts['jumlah_transaksi'], color='peru')
plt.title("Frekuensi Penggunaan Metode Pembayaran (Semua Tahun)")
plt.xlabel("Metode Pembayaran")
plt.ylabel("Jumlah Transaksi")
plt.tight_layout()
plt.show()


# Visualisasi: Pie Chart (presentase penggunaan)
plt.figure(figsize=(6,6))
plt.pie(payment_counts['jumlah_transaksi'],
        autopct='%1.1f%%', startangle=90)
plt.title("Presentase Penggunaan Metode Pembayaran")
plt.tight_layout()
plt.show()

# Analisis per tahun
yearly_payment = (
    df.groupby(['year', 'payment_method'])['total_price']
    .agg(['count', 'sum'])
    .reset_index()
    .sort_values(['year', 'count'], ascending=[True, False])
)

print("\n Ringkasasn Metode Pembayaran per Tahun:")
print(yearly_payment)


## 7. Langkah4: Membandingkan pola antara 2024 dan 2025.


**Apakah tren perilaku pelanggan berubah?**

In [ ]:
# Ringkasan performa per tahun
yearly_summary = (
    df.groupby('year')['total_price']
    .agg(['count', 'sum', 'mean'])
    .reset_index()
    .rename(columns={'count':'Jumlah Transaksi', 'sum':'Total Penjualan', 'mean':'Rata-rata Transaksi'})
)
print("Ringkasan Penjualan per Tahun:")
print(yearly_summary)

# Visualisasi total penjualan
plt.figure(figsize=(7,4))
plt.bar(yearly_summary['year'].astype(str), yearly_summary['Total Penjualan'], color='sienna')
plt.title("Total Penjualan per Tahun")
plt.xlabel("Tahun")
plt.ylabel("Total Penjualan")
plt.tight_layout()
plt.show()


plt.figure(figsize=(7,4))
plt.bar(yearly_summary['year'].astype(str), yearly_summary['Rata-rata Transaksi'], color='chocolate')
plt.title("Rata-rata Nilai Transaksi per Tahun")
plt.xlabel("Tahun")
plt.ylabel("Rata-rata Transaksi ($)")
plt.tight_layout()
plt.show()



